# Jupyter notebook to compute metrics for performance comparison
## importing packages and building dataset path

In [23]:
from pathlib import Path
import numpy
import glob
import torch
from skimage import metrics
from torchvision import transforms
from pytorch_msssim import ms_ssim
import cv2
from datetime import datetime
from numpy import mean
import lpips

device = 'cuda' if torch.cuda.is_available() else 'cpu'

loss_fn_alex = lpips.LPIPS(net='alex')

current_path = Path('.').resolve()
print(str(current_path))

reference_path = str(current_path)+'/IMAGES/PNG_IMAGES'
print("Reference source: ",reference_path)

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: C:\Users\filip\.conda\envs\ImageCompressionAI\Lib\site-packages\lpips\weights\v0.1\alex.pth
C:\Users\filip\DataspellProjects\ImageCompressionAI
Reference source:  C:\Users\filip\DataspellProjects\ImageCompressionAI/IMAGES/PNG_IMAGES


## Preparing vector of folders to compare

In [18]:
datasets = {
    'JPEG_1':'/IMAGES/JPEG/2',
    'JPEG_2':'/IMAGES/JPEG/6',
    'JPEG_3':'/IMAGES/JPEG/19',
    'JPEG_4':'/IMAGES/JPEG/23',
    'JPEG_5':'/IMAGES/JPEG/30',
    'JPEG2000_1':'/IMAGES/JPEG2000/175',
    'JPEG2000_2':'/IMAGES/JPEG2000/140',
    'JPEG2000_3':'/IMAGES/JPEG2000/115',
    'JPEG2000_4':'/IMAGES/JPEG2000/72',
    'JPEG2000_5':'/IMAGES/JPEG2000/56',
    'BPG_1':'/IMAGES/BPG/49',
    'BPG_2':'/IMAGES/BPG/44',
    'BPG_3':'/IMAGES/BPG/41',
    'BPG_4':'/IMAGES/BPG/38',
    'BPG_5':'/IMAGES/BPG/36',
    'VVC_1':'/IMAGES/VVC/42',
    'VVC_2':'/IMAGES/VVC/36',
    'VVC_3':'/IMAGES/VVC/33',
    'VVC_4':'/IMAGES/VVC/31',
    'VVC_5':'/IMAGES/VVC/30',
    'mbt2018_1':'/IMAGES/NET/mbt2018_1',
    'mbt2018_2':'/IMAGES/NET/mbt2018_2',
    'mbt2018_3':'/IMAGES/NET/mbt2018_3',
    'mbt2018_4':'/IMAGES/NET/mbt2018_4',
    'mbt2018_5':'/IMAGES/NET/mbt2018_5',
    'mbt2018_mean_1':'/IMAGES/NET/mbt2018_mean_1',
    'mbt2018_mean_2':'/IMAGES/NET/mbt2018_mean_2',
    'mbt2018_mean_3':'/IMAGES/NET/mbt2018_mean_3',
    'mbt2018_mean_4':'/IMAGES/NET/mbt2018_mean_4',
    'mbt2018_mean_5':'/IMAGES/NET/mbt2018_mean_5',
    'cheng2020_attn_1':'/IMAGES/NET/cheng2020_attn_1',
    'cheng2020_attn_2':'/IMAGES/NET/cheng2020_attn_2',
    'cheng2020_attn_3':'/IMAGES/NET/cheng2020_attn_3',
    'cheng2020_attn_4':'/IMAGES/NET/cheng2020_attn_4',
    'cheng2020_attn_5':'/IMAGES/NET/cheng2020_attn_5',
    'cheng2020-anchor_1':'/IMAGES/NET/cheng2020-anchor_1',
    'cheng2020-anchor_2':'/IMAGES/NET/cheng2020-anchor_2',
    'cheng2020-anchor_3':'/IMAGES/NET/cheng2020-anchor_3',
    'cheng2020-anchor_4':'/IMAGES/NET/cheng2020-anchor_4',
    'cheng2020-anchor_5':'/IMAGES/NET/cheng2020-anchor_5' 
}

## Computing metrics

In [30]:
for name, path in datasets.items():
    psnr_list=[]
    mssim_list=[]
    lpips_list=[]
    count = 0
    compressed_path = str(current_path)+path
    log = open(str(compressed_path)+'/metrics.txt', 'w')
    log.writelines(['Evaluating '+name+' with dataset: '+str(compressed_path) + ' on date:' + str(datetime.now()) + '\n'])
    print('Evaluating '+name+' with dataset: '+str(compressed_path))
    for file in glob.glob(str(compressed_path)+'/*.png'):
        reference_image = cv2.imread(reference_path+'/'+Path(file).stem+'.png',cv2.IMREAD_COLOR)
        reference = transforms.ToTensor()(reference_image).unsqueeze(0).to(device)
        reference_image = cv2.cvtColor(reference_image, cv2.COLOR_RGB2YCrCb)
        compressed_image = cv2.imread(file,cv2.IMREAD_COLOR)
        compressed = transforms.ToTensor()(compressed_image).unsqueeze(0).to(device)
        compressed_image = cv2.cvtColor(compressed_image, cv2.COLOR_RGB2YCrCb)
        OY, OCr, OCb = cv2.split(reference_image)
        Y, Cr, Cb = cv2.split(compressed_image)
        YMSE = metrics.mean_squared_error(OY,Y)
        CbMSE = metrics.mean_squared_error(OCb,Cb)
        CrMSE = metrics.mean_squared_error(OCr,Cr)
        MSE = (3/4)*YMSE + (1/8)*CbMSE + (1/8)*CrMSE
        psnr = 10 * numpy.log10((255*255) / MSE)
        psnr_list.append(psnr)
        mssim = ms_ssim(reference, compressed, data_range=1, size_average=True)
        mssim_list.append(mssim.item())
        d = loss_fn_alex(transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))(reference),transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))(compressed))
        lpips_list.append(d[0].item())
        log_string = str(Path(file).stem) + '\tPSNR: ' + str(psnr) + ' dB' + '\tMS-SSIM: '  + str(mssim.item()) + '\tLPIPS_AlexNet: ' + str(d[0].item()) + '\n'
        log.writelines([log_string])
        print(log_string)
        count += 1

    log.writelines(['\n',str('Average in ' + str(count) + ' images\nPSNR: ' + str(mean(psnr_list)) + 'dB\tMS-SSIM: ' + str(mean(mssim_list)) + '\tLPIPS_AlexNet: ' + str(mean(lpips_list)))])
    print(str('Average in ' + str(count) + ' images\nPSNR: ' + str(mean(psnr_list)) + 'dB\tMS-SSIM: ') + str(mean(mssim_list))  + '\tLPIPS_AlexNet: ' + str(mean(lpips_list)))
    log.close()

Evaluating JPEG_1 with dataset: C:\Users\filip\DataspellProjects\ImageCompressionAI/IMAGES/JPEG/2
IMG0001	PSNR: 25.021677948752092 dB	MS-SSIM: 0.7427448630332947	LPIPS_AlexNet: 0.6124216318130493


KeyboardInterrupt: 